In [ ]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetLarge, ResNet101, DenseNet121
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.metrics import Precision, Recall

from keras.models import load_model


In [ ]:
IMAGE_PATH = '../input/plant-pathology-2021-fgvc8/train_images'

In [ ]:
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train

In [ ]:
train['labels']=train['labels'].apply( lambda string: string.split(' ') )
train.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
label= mlb.fit_transform(train['labels'])
print(label)

In [ ]:
labels = pd.DataFrame(label,columns=mlb.classes_,index=train.index)
labels

In [ ]:
model = load_model('../input/notebooke6e8b09885/densenet.h5')

In [ ]:
test_path="../input/plant-pathology-2021-fgvc8/sample_submission.csv"
test = pd.read_csv(test_path)
test

In [ ]:
datagen = ImageDataGenerator(
    rescale=1/255
)

In [ ]:
test_data = datagen.flow_from_dataframe(
    test,
    directory='../input/plant-pathology-2021-fgvc8/test_images',
    x_col='image',
    y_col=None,
    color_mode='rgb',
    target_size=(256,256),
    class_mode=None,
    shuffle=False
)

In [ ]:
predictions = model.predict(test_data)
print(predictions)

In [ ]:
pred = (predictions>0.25)
pred

In [ ]:
for x in pred:
    count = 0
    for i in range(len(x)):
        if x[i]==False:
            count=count+1
    if count==len(x):
        x[2]=True

In [ ]:
for x in pred:
    if x[2]:
        for i in range(len(x)):
            x[i]=False
        x[2]=True

In [ ]:
pred

In [ ]:
label = labels.columns.tolist()[0:]
label

In [ ]:
pred_lists = []
for i in range(pred.shape[0]):
    l = []
    for j, c in enumerate(label):
        if pred[i, j]:
            l.append(c)
    pred_lists.append(l)

pred_lists = [' '.join(t) for t in pred_lists]
pred_lists

In [ ]:
test['labels'] = np.array(pred_lists)
test.head()

In [ ]:
test.to_csv('submission.csv', index=False)